In [1]:
import requests

In [2]:
url = "https://jsonplaceholder.typicode.com/todos/1"
response = requests.get(url=url)

print(f"Status code: {response.status_code}\n")
print(f"Message: {response.json()}")

Status code: 200

Message: {'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}
